In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
%run PARAMETERS.py
NAME = 'VAE'
PATH = 'VAE-200pts' 
CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:0')

Data_test = []
Ns = [120, 160, 200, 400, 600] 
for i in range(5):
    data = torch.from_numpy(np.load('/home/hao/Research/ncmm/rings_%d/' % (Ns[i]) + 'ob_%d.npy' % (Ns[i]))).float()
    Data_test.append(data)

In [3]:
from model_operations import Init_models_test

models = Init_models_test(K, 
                        D, 
                        hidden_list=HIDDEN_LIST, 
                        RECON_SIGMA=RECON_SIGMA, 
                        CUDA=CUDA, 
                        device=DEVICE, 
                        lr=LEARNING_RATE, 
                        RESTORE=True, 
                        PATH=PATH,
                        NAME=NAME)

In [4]:
from vae import VAE_test
MCMC_STEPS = 0
S = 500
B = 20
Test_Params = (K, S, B, DEVICE)
def Test_ALL(objective, data_g, mcmc_steps, Test_Params):
    Metrics = {'log_joint' : [], 'ess' : []}
    (K, S, B, DEVICE) = Test_Params
    NUM_DATASETS = data_g.shape[0]
    NUM_BATCHES = int((NUM_DATASETS / B))
    for step in range(NUM_BATCHES):
        ob = data_g[step*B : (step+1)*B]
        ob = ob.repeat(S, 1, 1, 1)
        ob = ob.cuda().to(DEVICE)
        metrics = VAE_test(models, ob, 0, K)
        Metrics['ess'].append(metrics['ess'] / S)
        Metrics['log_joint'].append(metrics['log_joint'])

    return Metrics

In [5]:
for g, data_g in enumerate(Data_test):
    time_start = time.time()
    %time Metrics = Test_ALL(VAE_test, data_g, MCMC_STEPS, Test_Params)
    time_end = time.time()
    print('group=%d (%ds)' % (g, (time_end - time_start)))
    for key in Metrics.keys():
        np.save('vae_%s-%s' % (Ns[g], key), torch.cat(Metrics[key], 0))

CPU times: user 9.36 s, sys: 2.56 s, total: 11.9 s
Wall time: 11.9 s
group=0 (11s)
CPU times: user 12.6 s, sys: 3.41 s, total: 16 s
Wall time: 16 s
group=1 (15s)
CPU times: user 15.3 s, sys: 4.32 s, total: 19.6 s
Wall time: 19.6 s
group=2 (19s)
CPU times: user 30.5 s, sys: 8.53 s, total: 39 s
Wall time: 39.1 s
group=3 (39s)
CPU times: user 46.6 s, sys: 14.6 s, total: 1min 1s
Wall time: 1min 1s
group=4 (61s)


In [ ]:
from testing import Eval
from viz_mc import Viz_MC
from apg import APG_test


BATCH_SIZE = 5
Viz_INTERVAL = 4
FS = 10
TITLE_FS = 20
OB_MS = 2.0
MU_MARKER = 'X'
MU_MS = 100
OPACITY = 1.0
BOUND = 10
COLORS = ['#EE7733', '#0077BB', '#AA3377', '#009988']
DATA_PTR = 3

Data = []

In [ ]:
evaluator = Eval(K, D, BATCH_SIZE, CUDA, DEVICE)
visualizer = Viz_MC(K, Viz_INTERVAL, FS, TITLE_FS, OB_MS, MU_MARKER, MU_MS, OPACITY, BOUND, COLORS)
# Metrics_sample = evaluator.Test_uniform(models, APG_test, Data, DATA_PTR, mcmc_steps=8, sample_size=1)

# visualizer.Plot_chains(Metrics_sample['data'], Metrics_sample['samples'], Metrics_sample['recon'], filename='ncmm-samples-%d' % DATA_PTR)

In [ ]:
L = 100
Metrics_converge = evaluator.Test_uniform(models, APG_test, Data, DATA_PTR, mcmc_steps=15, sample_size=L)
visualizer.Plot_metrics(Metrics_converge['ll'], Metrics_converge['ess'], sample_size=L, filename='ncmm-metrics-1')